### Imports inicales

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from IPython.display import display
import re

#Libreria para plotear
# %pip install --upgrade plotly
import plotly.express as px

# Librerias utiles
# %pip install --upgrade scikit-learn
from sklearn.preprocessing import StandardScaler

#pip install fuzzywuzzy


In [54]:
def eda(dataframe):
    pd.options.display.max_rows = 200
    pd.set_option('display.max_columns', None)

    print('1.- El DataFrame tiene', dataframe.shape[0],'filas y', dataframe.shape[1], 'columnas\n')
    print('2.- El DataFrame está compuesto por las siguientes columnas: ', list(dataframe.columns),'\n')
    print('3.- Ejemplos de filas del DataFrame:\n')
    
    print('Primeras 5 filas:')
    display(dataframe.head(5))
    print('Últimas 5 filas:')
    display(dataframe.tail(5))
    print('Muestreo aleatorio de 5 filas:')
    display(dataframe.sample(5))
    print('4.- Descripción numérica del Dataframe:\n')
    display(dataframe.describe())
    print('5.- Cantidad de valores nulos por columna:')
    display(dataframe.isna().sum())
    print('6.- Cantidad de valores únicos por columna:')
    unique_values = dataframe.nunique()
    display(unique_values)
    print('7.- Filas totales duplicadas en el DataFrame:')
    display(dataframe.duplicated().sum())
    print('8.- Filas duplicadas en el DataFrame:')
    display(dataframe[dataframe.duplicated(keep=False)])
    print('9.- Descripción tipos de datos del Dataframe:\n')
    display(dataframe.dtypes)

In [3]:
#restaurantes = pd.read_csv("./restaurants.csv")
#menus = pd.read_csv("./restaurant-menus.csv")

### Nexrt steps:
- Preprocesamiento: Limpiar la data, quitar columnas innecesarias y nulos. Cambiar precios a números
- Definir estados a trabajar
- Nueva columna de franquisia acsociada: Se contrulle dándole un diccionario y ver si el nombre calza con un nombre de cadena.
- Usar mismo csv para mabas cosas



Nos vamos aquedar ocn datos de ciertos estados

- Vsualización PowerBi:

 Mapa de algo, 
Comparar precio de la misma categorpia entre restaruantes vs sus rating
Puntación vs precio


Catgorías de restaruantes que más se repiten por estado
N° de franquisias por estado
Precio de la misma franquisia entre estados


-D3:
- Gráfico de barras pra comprar precios entre estados de la misma categoria
- Grafico de barras con cadenas por estado y al poner mouse encima aparace distribución de franquicias

La siguiete parte procesa los dos DF originales y deja solo los datos de cadenas de comida rápida relevantes que queremos usar:

In [6]:

# Esta parte quita las direcciones delos nombres de las cadenas

# Cargar el DataFrame 'restaurantes' (asumiendo que ya lo has cargado)
# restaurantes = pd.read_csv('ruta_a_tu_archivo.csv')

# Crear una copia del DataFrame para trabajar con ella
df_copia0 = restaurantes.copy()

# Función para eliminar texto entre paréntesis
def eliminar_direccion(nombre):
    # Elimina el texto entre paréntesis y los paréntesis mismos
    return re.sub(r'\(.*?\)', '', nombre).strip()

# Aplicar la función para limpiar los nombres
df_copia0['nombre_limpio'] = df_copia0['name'].apply(eliminar_direccion)

# Contar la frecuencia de cada nombre limpio
frecuencias = df_copia0['nombre_limpio'].value_counts()

# Filtrar los nombres que se repiten 3 veces o más
nombres_repetidos = frecuencias[frecuencias >= 3].index

# Quedarse solo con los nombres que se repiten 3 veces o más
df_sin_direccion = df_copia0[df_copia0['nombre_limpio'].isin(nombres_repetidos)]

# Mostrar las primeras filas del DataFrame final para verificar los resultados
df_sin_direccion



# Esto crea la columna nombre_caednas y le asigna el nombre de la cadena que esté en el top50, al resto lo deja en nulo

# Lista de cadenas
cadenas = [
    'Subway', 'Starbucks', 'McDonald’s', 'Dunkin’', 'Burger King', 'Taco Bell',
    'Pizza Hut', 'Domino’s', 'Wendy’s', 'Dairy Queen', 'Little Caesars', 'KFC',
    'Sonic Drive-In', 'Arby’s', 'Papa John’s', 'Chipotle', 'Jimmy John’s', 
    'Popeyes Louisiana Chicken', 'Chick-fil-A', 'Baskin-Robbins', 'Panda Express', 
    'Jack in the Box', 'Panera Bread', 'Jersey Mike’s', 'Hardee’s', 'Wingstop', 
    'Five Guys', 'Papa Murphy’s', 'Firehouse Subs', 'Carl’s Jr.', 'Marco’s Pizza', 
    'Tropical Smoothie Cafe', 'Church’s Chicken', 'Zaxby’s', 'Checkers and Rally’s', 
    'Whataburger', 'Culver’s', 'Bojangles', 'Qdoba', 'Moe’s', 'Tim Hortons', 
    'Del Taco', 'Steak ‘n Shake', 'Raising Cane’s', 'El Pollo Loco', 'McAlister’s Deli', 
    'Freddy’s Frozen Custard & Steakburgers', 'Krispy Kreme', 'White Castle', 
    'In-N-Out Burger'
]

# Normalización de cadenas
def normalizar(texto):
    texto = texto.lower()  # Convertir a minúsculas
    texto = re.sub(r'[^\w\s]', '', texto)  # Eliminar signos de puntuación
    texto = texto.replace(' ', '')  # Eliminar espacios
    return texto

# Normalizar la lista de cadenas
cadenas_normalizadas = {normalizar(cadena): cadena for cadena in cadenas}

# Crear una copia del DataFrame para trabajar con ella
df_copia1 = df_sin_direccion.copy()

# Seleccionar solo las columnas 'id', 'nombre_limpio' y 'name'
df_copia1 = df_copia1[['id', 'nombre_limpio', 'name']]

# Normalizar la columna 'nombre_limpio'
df_copia1['nombre_limpio_normalizado'] = df_copia1['nombre_limpio'].apply(normalizar)

# Función para asignar el nombre de la cadena basado en la normalización
def asignar_nombre_cadena(nombre_normalizado):
    if nombre_normalizado in cadenas_normalizadas:
        return cadenas_normalizadas[nombre_normalizado]
    return None

# Crear la nueva columna 'nombre_cadena' aplicando la función
df_copia1['nombre_cadena'] = df_copia1['nombre_limpio_normalizado'].apply(asignar_nombre_cadena)

# Eliminar la columna de normalización adicional si ya no es necesaria
df_copia1 = df_copia1.drop(columns=['nombre_limpio_normalizado'])

# Mostrar las primeras filas del DataFrame copiado para verificar los resultados
df_copia1



# Filtrar las filas donde 'nombre_cadena' es nulo
df_nulos = df_copia1[df_copia1['nombre_cadena'].isna()]

# Contar las apariciones de cada valor en 'nombre_limpio'
conteo_nombre_limpio = df_nulos['nombre_limpio'].value_counts()

# Filtrar para obtener solo los nombres que aparecen más de 100 veces
nombres_mas_de_100 = conteo_nombre_limpio[conteo_nombre_limpio > 100]

# Convertir a una lista si es necesario
nombres_mas_de_100_lista = nombres_mas_de_100.index.tolist()



# Crear un diccionario para mapear nombres frecuentes a ellos mismos (o nombres de cadenas si ya se sabe)
# En este caso, el nombre del mapeo es el mismo ya que estamos reemplazando por el mismo valor
df_copia2 = df_copia1.copy()

nombre_mapeo = {nombre: nombre for nombre in nombres_mas_de_100_lista}

# Actualizar 'nombre_cadena' en df_copia2 usando el diccionario para filas que coinciden con nombres_mas_de_100
df_copia2['nombre_cadena'] = df_copia2.apply(
    lambda row: nombre_mapeo.get(row['nombre_limpio'], row['nombre_cadena']),
    axis=1
)


# Lista de nombres de establecimientos que no son comida rápida
no_comida_rapida = [
    'The Ice Cream Shop', 'Cracker Barrel Old Country Store', 'Outback Steakhouse',
    'Walgreens', 'Bed Bath & Beyond', 'CVS', 'Safeway Express', 'ProFlowers', 'Rite Aid', '7-Eleven'
]

# Filtrar el DataFrame para eliminar las filas con esos nombres en 'nombre_cadena'
df_copia3 = df_copia2[~df_copia2['nombre_cadena'].isin(no_comida_rapida)]

# Realizar la fusión de los DataFrames basado en la columna 'id'
df_filtrado = pd.merge(restaurantes, df_copia3[['id', 'nombre_cadena']], on='id', how='inner')

# Filtrar las filas donde 'nombre_cadena' no es nulo
df_no_null = df_filtrado[df_filtrado['nombre_cadena'].notna()]

# Reordenar las columnas para que 'nombre_cadena' sea la cuarta columna
# Primero obtenemos el nombre de las columnas
cols = df_no_null.columns.tolist()

# Aseguramos que 'nombre_cadena' está en la lista de columnas
if 'nombre_cadena' in cols:
    # Reordenamos las columnas para que 'nombre_cadena' sea la cuarta
    cols.remove('nombre_cadena')
    cols.insert(3, 'nombre_cadena')  # Insertar 'nombre_cadena' en la cuarta posición
    df_no_null = df_no_null[cols]


# Filtrar menus para conservar solo las filas con ids que están en df_no_null
df_filtrado_menus = menus[menus['restaurant_id'].isin(df_no_null['id'])]


# Exportar el DataFrame filtrado a un archivo CSV
#df_no_null.to_csv('cadenas.csv', index=False)

#df_filtrado_menus.to_csv('menus-cadenas.csv', index=False)

### Ahora limpaimos la data

In [68]:
cadenas = pd.read_csv("./cadenas.csv")
menuCadenas = pd.read_csv("./menus-cadenas.csv")

#### Limpieza cadenas

In [69]:
cadenas['full_address'] = cadenas['full_address'].astype(str)

## Trabajo con zip_code

# Primero, corregimos la fila 8651 cambiando el valor de zip_code de 78749.0 a 78749
cadenas.loc[8651, 'zip_code'] = 78749
# Ahora, eliminamos las filas donde zip_code sea igual a 'Milwaukee' o 'US'
cadenas = cadenas[~cadenas['zip_code'].isin(['Milwaukee', 'US'])]
# Convierte 'zip_code' a cadena para asegurar que str.contains funcione
cadenas['zip_code'] = cadenas['zip_code'].astype(str)
# Filtra las filas donde 'zip_code' no contenga '-'
cadenas = cadenas[~cadenas['zip_code'].str.contains('-')]
# Convierte 'zip_code' de nuevo a números (puede ser int o float dependiendo de los datos)
cadenas['zip_code'] = pd.to_numeric(cadenas['zip_code'], errors='coerce')

# Función para extraer el estado o la ciudad del formato de dirección
def extraer_estado(direccion):
    partes = direccion.split(',')
    
    # Si hay al menos tres partes (incluyendo la ciudad y el estado)
    if len(partes) >= 3:
        estado = partes[2].strip()
        # Si el estado está vacío, toma la ciudad
        if estado == '':
            return partes[1].strip()  # Devuelve la ciudad si el estado está vacío
        return estado
    elif len(partes) >= 2:
        # Si no hay suficientes partes para el estado, devuelve la ciudad
        return partes[1].strip()

    return pd.NA  # Si no se puede determinar ni estado ni ciudad

# Aplicar la función para crear una nueva columna 'estado'
cadenas['estado'] = cadenas['full_address'].apply(extraer_estado)

# Diccionario con los reemplazos de los valores incorrectos por su estado correspondiente
reemplazos_estado = {
    'Fultondale': 'AL',
    'Birmingham': 'AL',
    'Huntsville': 'AL',
    'USA': pd.NA,  # Valor a eliminar
    'Howard': 'WI',
    'Janesville': 'WI',
    'Martinsburg': 'WV',
    'Marysville': 'WA',
    'Lynnwood': 'WA',
    'Covington': 'VA',
    'Alexandria': 'VA',
    'Washington': 'DC',
    'Virginia Beach': 'VA',
    'Roanoke': 'VA',
    'Herndon': 'VA',
    'Reston': 'VA',
    'Sterling': 'VA',
    'Ashburn': 'VA',
    'Arlington': 'VA',
    'FREDERICKSBURG': 'VA',
    'Lynchburg': 'VA',
    'Chesapeake': 'VA',
    'Old Town Manassas': 'VA',
    'Blacksburg': 'VA',
    'West Jordan': 'UT',
    'South Jordan': 'UT',
    'West Valley City': 'UT',
    'San Juan': 'PR',
    'Carolina': 'PR',
    'Bayamón': 'PR',
    'Toa Baja': 'PR',
    'Dorado': 'PR',
    'San Antonio': 'TX',
    'Selma': 'TX',
    'Texas': 'TX',
    'Katy': 'TX',
    'Beaumont': 'TX',
    'McKinney': 'TX',
    'Carrolton': 'TX',
    'Dallas': 'TX',
    'Sugar Land': 'TX',
    'Pearland': 'TX',
    'Humble': 'TX',
    'New Caney': 'TX',
    'Deer Park': 'TX',
    'La Porte': 'TX',
    'Harker Heights': 'TX',
    'Round Rock': 'TX',
    'Plano': 'TX',
    'Plugerville': 'TX',
    'Baytown': 'TX'
}

# Reemplazar los valores incorrectos en la columna 'estado'
cadenas['estado'] = cadenas['estado'].replace(reemplazos_estado)

# Eliminar filas que tengan NaN en columnas clave (por ejemplo, 'estado' y otras)
cadenas = cadenas.dropna()

# Convertir columnas a los tipos de datos adecuados
cadenas['id'] = cadenas['id'].astype(int)
cadenas['position'] = cadenas['position'].astype(int)
cadenas['score'] = cadenas['score'].astype(float)
cadenas['ratings'] = cadenas['ratings'].astype(int)
cadenas['zip_code'] = cadenas['zip_code'].astype(int)
cadenas['estado'] = cadenas['estado'].astype(str)


# Creamos nueva columna con lista de categrias del restaurante
# Asegúrate de que la columna 'category' es de tipo cadena
cadenas['category'] = cadenas['category'].astype(str)

# Convierte la columna 'category' en una lista de listas
# Cada fila se convierte en una lista de categorías
cadenas['category_list'] = cadenas['category'].apply(lambda x: [cat.strip() for cat in x.split(',')])

# Si deseas obtener una lista única de todas las categorías sin repeticiones
# Primero, obtén todas las categorías en una sola lista
all_categories = [cat for sublist in cadenas['category_list'] for cat in sublist]

# Luego, elimina las duplicaciones para obtener una lista única de categorías
unique_categories = list(set(all_categories))

In [58]:
# Verifica los resultados
print(cadenas[['category', 'category_list']].head())
unique_categories

                                            category  \
1  Breakfast and Brunch, salad, Sandwich, Family ...   
3                        American, Burgers, Sandwich   
4  Bakery, Breakfast and Brunch, Cafe, Coffee &am...   
5            Sandwich, Salads, Healthy, Family Meals   
6                                     Sandwich, Deli   

                                       category_list  
1  [Breakfast and Brunch, salad, Sandwich, Family...  
3                      [American, Burgers, Sandwich]  
4  [Bakery, Breakfast and Brunch, Cafe, Coffee &a...  
5          [Sandwich, Salads, Healthy, Family Meals]  
6                                   [Sandwich, Deli]  


['Family Meals',
 'Mexican',
 'Asian',
 'American (Traditional)',
 'Italian',
 'Diner',
 'Specialty Foods',
 'Dinner',
 'Pasta',
 'Sandwiches',
 'Juice &amp; Smoothies',
 'Ice Cream &amp; Frozen Yogurt',
 'Healthy',
 'Tex Mex',
 'Salads',
 'Jamaican',
 'Burgers',
 'Omelette',
 'Hot Dog',
 'Brunch',
 'BBQ',
 'Pizza',
 'Cantonese',
 'Allergy Friendly',
 'Sandwich',
 'Vegetarian-Friendly',
 'Latin Fusion',
 'Fast Food',
 'New Mexican',
 'Sandwhiches',
 'Bowls',
 'Exclusive to Eats',
 'Chicken',
 'Coffee &amp; Tea',
 'Tacos',
 'Sports Bar',
 'Ice Cream + Frozen Yogurt',
 'pizza',
 'Black-owned',
 'Desserts',
 'Group Friendly',
 'Fried Chicken',
 'Vegetarian Friendly',
 'Burritos',
 'Pastry',
 'American',
 'Soul Food',
 'Sanwiches',
 'whatever',
 'Deli',
 'Donuts',
 'Bar Food',
 'Southern',
 'Ice Cream',
 'Juice and Smoothies',
 'pasta',
 'Seafood',
 'Western',
 'JuiceAndSmoothie',
 'Coffee and Tea',
 'wings',
 'PIzza',
 'South American',
 'Kids Friendly',
 'Bakery',
 'Vegetarian',
 'Comfor

In [55]:
eda(cadenas.drop(columns=['category_list'], errors='ignore'))

1.- El DataFrame tiene 10221 filas y 13 columnas

2.- El DataFrame está compuesto por las siguientes columnas:  ['id', 'position', 'name', 'nombre_cadena', 'score', 'ratings', 'category', 'price_range', 'full_address', 'zip_code', 'lat', 'lng', 'estado'] 

3.- Ejemplos de filas del DataFrame:

Primeras 5 filas:


,id,position,name,nombre_cadena,score,ratings,category,price_range,full_address,zip_code,lat,lng,estado
1,14,51,Panera (521 Fieldstown Road),Panera,4.6,44,"Breakfast and Brunch, salad, Sandwich, Family ...",$,"521 Fieldstown Road, Gardendale, AL, 35071",35071,33.651407,-86.819247,AL
3,20,40,MrBeast Burger (838 Odum Road),MrBeast Burger,3.7,19,"American, Burgers, Sandwich",$$,"838 Odum Road, Gardendale, AL, 35071",35071,33.645480,-86.826260,AL
4,35,80,Starbucks (Hwy 11 and Avenue W),Starbucks,4.6,21,"Bakery, Breakfast and Brunch, Cafe, Coffee &am...",$,"2401 Crossplex Boulevard 101, Birmingham, AL, ...",35208,33.497944,-86.874391,AL
5,41,15,McAlister's Deli (1801 4th Avenue South),McAlister’s Deli,4.7,100,"Sandwich, Salads, Healthy, Family Meals",$,"1801 4th Ave S, Birmingham, AL, 35233",35233,33.507150,-86.803990,AL
6,42,13,"Firehouse Subs (3477 Lowery Parkway, Suite 115)",Firehouse Subs,4.7,23,"Sandwich, Deli",$,"3477 Lowery Parkway, Suite 115, Fultondale, A...",35068,33.605213,-86.805621,AL


Últimas 5 filas:


,id,position,name,nombre_cadena,score,ratings,category,price_range,full_address,zip_code,lat,lng,estado
13541,63443,205,KFC (707 Nolana),KFC,4.8,28,"American, Fast Food, wings, Family Meals",$,"707 Nolana, McAllen, TX, 78504",78504,26.238411,-98.220305,TX
13542,63444,200,Chick-fil-A (2709 W Nolana Ave),Chick-fil-A,4.9,100,"American, Fried Chicken, Family Meals, Souther...",$,"2709 W Nolana Ave, Mcallen, TX, 78504",78504,26.240373,-98.245054,TX
13543,63463,41,Whataburger (4017 N 23rd St),Whataburger,5.0,26,"Burgers, Fast Food, Burgers, Sandwiches, American",$$,"4017 N 23rd St, McAllen, TX, 78504",78504,26.242120,-98.239670,TX
13544,63465,2,IHOP (4321 N 10th St),IHOP,4.6,56,"American, Breakfast and Brunch, burger, Comfor...",$,"4321 N 10th St, MC ALLEN, TX, 78504",78504,26.242853,-98.223984,TX
13545,63468,19,Church's Chicken (620 S. 10th Street),Church’s Chicken,4.3,17,"Fast Food, Family Meals",$,"620 S. 10th Street, McAllen, TX, 78501",78501,26.197333,-98.231369,TX


Muestreo aleatorio de 5 filas:


...

np.int64(0)

8.- Filas duplicadas en el DataFrame:


,id,position,name,nombre_cadena,score,ratings,category,price_range,full_address,zip_code,lat,lng,estado


9.- Descripción tipos de datos del Dataframe:



id                 int64
position           int64
name              object
nombre_cadena     object
score            float64
ratings            int64
category          object
price_range       object
full_address      object
zip_code           int64
lat              float64
lng              float64
estado            object
dtype: object

#### Limpieza Menu Cadenas

In [70]:
# Función para limpiar y convertir el precio a numérico
def limpiar_precio(precio):
    try:
        # Eliminar " USD" y convertir a número
        return float(precio.replace(' USD', '').replace(',', '.'))
    except:
        return None

# Aplicar la función para transformar la columna 'price'
menuCadenas['price'] = menuCadenas['price'].apply(limpiar_precio) 

# Eliminados duplicados
menuCadenas = menuCadenas.drop_duplicates()

In [71]:
# Obtener los IDs de 'cadenas'
ids_cadenas = cadenas['id'].unique()

# Filtrar 'menuCadenas' para mantener solo las filas con IDs en 'ids_cadenas'
menuCadenas = menuCadenas[menuCadenas['restaurant_id'].isin(ids_cadenas)]

In [72]:
eda(menuCadenas)

1.- El DataFrame tiene 1010426 filas y 5 columnas

2.- El DataFrame está compuesto por las siguientes columnas:  ['restaurant_id', 'category', 'name', 'description', 'price'] 

3.- Ejemplos de filas del DataFrame:

Primeras 5 filas:


,restaurant_id,category,name,description,price
37,14,Picked for you,Frozen Strawberry Lemonade,"130 Cal. Strawberry fruit blend, our Agave Lem...",4.19
38,14,Picked for you,Broccoli Cheddar Soup,"Bowl (360 Cal.), Cup (230 Cal.), Bread Bowl (9...",6.49
39,14,Picked for you,NEW Mango Yuzu Citrus - With Guarana &amp; Caf...,"Regular (330 Cal.), Large (500 Cal.) Signature...",4.59
40,14,Picked for you,Caesar Salad with Chicken,"Whole (440 Cal.), Half (220 Cal.) Chicken rais...",8.79
41,14,Picked for you,Pepperoni Flatbread Pizza,"980 Cal. Thick sliced Pepperoni, fresh mozzare...",11.49


Últimas 5 filas:


,restaurant_id,category,name,description,price
1390363,63468,Homestyle Sides,Baked Mac &amp; Cheese,"We take mac &amp; cheese, sprinkle shredded ch...",3.29
1390364,63468,Homestyle Sides,Corn,"Sweet, buttery and bursting with flavor. Our c...",3.29
1390365,63468,Homestyle Sides,Jalapeño Cheese Bombers®,Fried spicy jalapeño bits and creamy cheddar c...,3.29
1390366,63468,Desserts &amp; More,Apple Pies,Satisfy your sweet tooth with our apple pie. J...,1.69
1390367,63468,Desserts &amp; More,Frosted Biscuits,NaN,1.49


Muestreo aleatorio de 5 filas:


...

np.int64(0)

8.- Filas duplicadas en el DataFrame:


,restaurant_id,category,name,description,price


9.- Descripción tipos de datos del Dataframe:



restaurant_id      int64
category          object
name              object
description       object
price            float64
dtype: object

#### Exportamos a nuevo dataframes a usar en las visualizaciones


In [74]:
#menuCadenas.to_csv('menuCadenasClean.csv', index=False)
#cadenas.to_csv('cadenasClean.csv', index=False)